## Create tri-colored PNGs for all slices in HDF5s

In [1]:
import os, sys, re, psycopg2, subprocess
import numpy as np
from subprocess import call
from natsort import natsorted

#### Set context

In [10]:
stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines"
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"

os.chdir(stack_folder)
cwd = os.getcwd()
print("Current working directory %s" % cwd)

Current working directory /Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines


#### Get stack names

In [11]:
stacks = sorted([d for d in os.listdir(stack_folder) if not d.startswith('.')])
print(len(stacks), stacks)

309 ['0-Reference-Brain', '6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2', 'Anti-tERK', 'Elavl3-GCaMP5G', 'Elavl3-H2BRFP', 'EtVmat2-GFP', 'Gad1b-GFP', 'Glyt2-GFP', 'HGn39D', 'Hcrt-RFP', 'Isl1-GFP', 'Isl2bGal4-uasDendra', 'MH_13319-Gal4', 'MH_15576-Gal4', 'MH_16659-Gal4', 'MH_16715-Gal4', 'MH_16913-Gal4', 'MH_16914-Gal4', 'MH_16915-Gal4', 'MH_16916-Gal4', 'MH_16946-Gal4', 'MH_17056-Gal4', 'MH_17767-Gal4', 'MH_17768-Gal4', 'MH_17776-Gal4', 'MH_17777-Gal4', 'MH_17787-Gal4', 'MH_17845-Gal4', 'MH_17846-Gal4', 'MH_18107-Gal4', 'MH_18108-Gal4', 'MH_18327-Gal4', 'MH_18329-Gal4', 'MH_19322-Gal4', 'MH_19566-Gal4', 'MH_20360-Gal4', 'MH_20361-Gal4', 'MH_20454-Gal4', 'MH_20552-Gal4', 'MH_20553-Gal4', 'MH_20638-Gal4', 'MH_20639-Gal4', 'MH_20640-Gal4', 'MH_20641-Gal4', 'MH_20642-Gal4', 'MH_20643-Gal4', 'MH_20644-Gal4', 'MH_20646-Gal4', 'MH_20647-Gal4', 'MH_20648-Gal4', 'MH_20649-G

In [12]:
# Optional, for selecting specific stacks
selector = "HGn39D"
stacks = [stack for stack in stacks if selector in stack]
print(len(stacks), stacks)

1 ['HGn39D']


#### Create color channel folders & color subfolders

In [13]:
colors = ['red', 'blue', 'green', 'cyan']
new_stacks = []

for stack in stacks:
    folder_path = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    try:
        os.mkdir(folder_path)
        new_stacks.append(stack)
    except:
        pass
    finally:
        for color in colors:
            channel_path = folder_path + color
            try:
                os.mkdir(channel_path)
            except:
                pass

#### Create color channel overlays for each stack (or each new stack depending on input)

In [6]:
for i, stack in enumerate(stacks):
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    
    os.chdir(src)
    for color in colors:
        if color is 'green':
            color_val = 'green1'
            fuzz = '7%'
        else:
            color_val = color
            fuzz = '9%'
            
        dest = base + color
        cwd = os.getcwd()
        print(cwd)
        try:
            subprocess.check_output(["mogrify", "-path", dest, "-format", "png", "+level-colors", "black,%s"%color_val, "-fuzz", fuzz, "-transparent", "black", "*.jpg"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)


/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/HGn39D
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/HGn39D
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/HGn39D
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/HGn39D


### Prepare PostgreSQL table

In [2]:
create_table_command = (
"""
    DROP TABLE colorchannels;
    CREATE TABLE colorChannels (
        channel_image_id SERIAL PRIMARY KEY,
        channel_name TEXT NOT NULL,
        channel_id INT NOT NULL,
        channel_image_path TEXT NOT NULL
    );
""")

#### Connect to zbrain2db database and create table


In [3]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [4]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [5]:
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"
stacks = sorted([d for d in os.listdir(channel_folder) if not d.startswith('.')])
print(stacks[:5])

['0-Reference-Brain', '6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR']


#### Create list of table insert commands for each color channel image

In [6]:
def create_insert_commands(stacks):
    stack_id, image_id = 0, 0
    for stack in stacks:
        images_folder = os.path.join('/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/', stack, 'red')
        images = natsorted([c for c in os.listdir(images_folder) if not c.startswith('.')])
        
        for image_path in images:
            insert_command = ("INSERT INTO colorChannels (channel_image_id, channel_name, channel_id, channel_image_path) values ({0}, '{1}', {2}, '{3}')" 
                                 .format(image_id, stack, stack_id, image_path))
            yield insert_command
            image_id += 1
            
        stack_id += 1

In [7]:
insert_commands = create_insert_commands(stacks)

#### Iterate insert commands list to create table entries for each color channel image

In [8]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

#### Query updated table

In [11]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM colorChannels')

rows = cursor.fetchall()
for row in rows[0:1000]:
    print(row)

(0, '0-Reference-Brain', 0, 'Reference-Brain-0.png')
(1, '0-Reference-Brain', 0, 'Reference-Brain-1.png')
(2, '0-Reference-Brain', 0, 'Reference-Brain-2.png')
(3, '0-Reference-Brain', 0, 'Reference-Brain-3.png')
(4, '0-Reference-Brain', 0, 'Reference-Brain-4.png')
(5, '0-Reference-Brain', 0, 'Reference-Brain-5.png')
(6, '0-Reference-Brain', 0, 'Reference-Brain-6.png')
(7, '0-Reference-Brain', 0, 'Reference-Brain-7.png')
(8, '0-Reference-Brain', 0, 'Reference-Brain-8.png')
(9, '0-Reference-Brain', 0, 'Reference-Brain-9.png')
(10, '0-Reference-Brain', 0, 'Reference-Brain-10.png')
(11, '0-Reference-Brain', 0, 'Reference-Brain-11.png')
(12, '0-Reference-Brain', 0, 'Reference-Brain-12.png')
(13, '0-Reference-Brain', 0, 'Reference-Brain-13.png')
(14, '0-Reference-Brain', 0, 'Reference-Brain-14.png')
(15, '0-Reference-Brain', 0, 'Reference-Brain-15.png')
(16, '0-Reference-Brain', 0, 'Reference-Brain-16.png')
(17, '0-Reference-Brain', 0, 'Reference-Brain-17.png')
(18, '0-Reference-Brain', 0, '